# Overview


For the molecule lovers in the audience, let's calculate a transition state for the C-H bond dissociation of methane (CH4). For this exercise, we will use a semi-empirical quantum chemistry method called GFN1-xTB as our calculator (https://pubs.acs.org/doi/abs/10.1021/acs.jctc.8b01176). We will cover such methods later, and it is not the focal point for now.

Note: We are not using the newer GFN2-xTB because for some reason, the transition state search does not identify a reasonable structure with that method. Beats me, but we move on.


# Setup


To install the dependencies, please run

Mac/Linux:

```bash
uv pip install ase tblite sella
```

Windows:

```bash
uv pip install ase sella
conda install -c conda-forge tblite-python
```


# Demonstration


We start by making the CH4 molecule. You can look up the XYZ coordinates of the molecule or just use the built-in one in ASE.


In [ ]:
from ase.build import molecule

ch4 = molecule("CH4")

Let's make sure to relax the geometry to have a local minimum energy structure to start.


In [ ]:
from ase.optimize import BFGS
from tblite.ase import TBLite

initial = ch4.copy()
initial.calc = TBLite(method="GFN1-xTB", verbosity=0)
opt = BFGS(initial)
opt.run(fmax=0.01)

Now we need a guess for the product state, which should be a CH3 radical and H radical separated from one another. We can achieve this by increase a C-H bond distance to some large value like 5.0 A.


In [ ]:
final = initial.copy()
final.set_distance(0, 1, 5.0, fix=0)  # 0 = C, 1 = H

Let's make sure we made what we think we've made.


In [ ]:
from ase.visualize import view

view(final)

Now let's optimize the product state. We will discuss this more later, but we need to be careful about spin. CH3 and H in the dissociated product are both radical species, meaning they each have one unpaired electron. Electronic structure codes, including some semi-empirical quantum chemistry codes and even some machine learning potentials, take the number of unpaired electrons (and charge) as an input parameter. Here, multiplicity is simply the number of unpaired electrons + 1, which would be 3 for the product state. If you did not set `multiplicity=3`, the calculation would crash and tell you it could not converge.


In [ ]:
final.calc = TBLite(method="GFN1-xTB", multiplicity=3, verbosity=0)
opt = BFGS(final)
opt.run(fmax=0.01)

The NEB method is really difficult to use in cases where the spin state can change between the reactants and products because the NEB method generally keeps the spin state fixed through the course of the MEP search process even if the images might become more reactant- or more product-like.


Since this is a simple reaction, we can manually scan the reaction coordinate and identify the structure associated with the highest energy configuration. We will linearly interpolate using the NEB module for convenience but won't use the NEB method itself here.


In [ ]:
from ase.mep import NEB

images = [initial] + [initial.copy() for _ in range(10)] + [final]
neb = NEB(images, method="improvedtangent")
neb.interpolate()

In [ ]:
view(images)

Now we will calculate the energy of each intermediate image. This is called a static scan because we are not relaxing the intermediate images at all. This is not a true MEP we are mapping, but it will give us a reasonable start.

Note that I have forced each intermediate image to have `multiplicity=3`. This was based on personal experience. You could tinker with this since at some point along the MEP, it must cross from `multiplicity=1` (initial state) and `multiplicity=3` (final state).


In [ ]:
for i, image in enumerate(images[1:-1], start=1):
    image.calc = TBLite(method="GFN1-xTB", multiplicity=3, verbosity=0)
    e = image.get_potential_energy()
    print(f"Image {i}: Energy = {e} eV")

Great. Now let's plot the results. It does not make sense to plot absolute energies, so we will normalize everything by the lowest energy structure (the reactant). We will also plot the energy as a function of our reaction coordinate, which is the C-H bond distance.


In [ ]:
from matplotlib import pyplot as plt

energies = [image.get_potential_energy() for image in images]
energies = [e - min(energies) for e in energies]
ch_bond_dist = [image.get_distance(0, 1) for image in images]

plt.plot(ch_bond_dist, energies, marker="o")
plt.ylabel("Energy (eV)")
plt.xlabel("C-H Bond Distance (Å)")

From here, we have a decent estimate of the activation energy (and an unambiguous determination of the reaction energy).


In [ ]:
barrier = max(energies) - energies[0]
rxn_energy = energies[-1] - energies[0]
print(f"Activation Barrier: {barrier * 96.49} kJ/mol")
print(f"Reaction Energy: {rxn_energy * 96.49} kJ/mol")

The experimental values are known. For instance, the kinetics can be found at https://kinetics.nist.gov/kinetics and the thermodynamics in the CRC Handbook.

The values are way off. The bond-dissociation enthalpy of methane is 439 kJ/mol. Granted, this is enthalpy at room temperature and we are only reporting the electronic (0 K) energy, but the enthalpic corrections are very small.

It turns out the the semi-empirical GFN-xTB methods are pretty bad at predicting energies, including both GFN1-xTB and GFN2-xTB. In fact, the name suggests what they are good for: geometries, frequencies, and non-covalent interactions. But it was fast, so we used it here anyway. The same exercise could be done with a more accurate calculator.


The example above was a static scan, and there is no guarantee that the highest energy structure was _exactly_ the transition state. We can take the highest energy image and pass it to a saddle point search algorithm. A fancy one is called Sella (https://pubs.acs.org/doi/full/10.1021/acs.jctc.2c00395).


First, we identify the highest energy structure and use that as our guess for the transition state.


In [ ]:
import numpy as np

ts_idx = np.argmax(energies)
ts_guess = images[ts_idx]

In [ ]:
view(ts_guess)

Then we run a typical ASE optimization except here the optimizer is `Sella` instead of something like `BFGS`. We do not know in advance if the transition state has no unpaired electrons (like the reactant) or 2 unpaired electrons (like the products), so we will try both and look at the resulting structures. It will be pretty obvious which is really the transition state.

Actually, upon running this, we find that the `multiplicity=1` case does not even converge at all, so that settles it.


In [ ]:
ts_guess1 = ts_guess.copy()
ts_guess2 = ts_guess.copy()
ts_guess1.calc = TBLite(method="GFN1-xTB", multiplicity=1, verbosity=0)
ts_guess2.calc = TBLite(method="GFN1-xTB", multiplicity=3, verbosity=0)

In [ ]:
from sella import Sella

# order = 1 for TS; order = 0 for geometry opt
# internal = True to use internal coordinates for speedup; only works for molecules
try:
    opt = Sella(ts_guess1, trajectory="ts.traj", order=1, internal=True)
    opt.run(fmax=0.01)
except Exception:
    print("Spin multiplicity 1 optimization failed")

try:
    opt = Sella(ts_guess2, trajectory="ts2.traj", order=1, internal=True)
except Exception:
    print("Spin multiplicity 3 optimization failed")
opt.run(fmax=0.01)

It is pretty clear that the TS is the second one (`multiplicity=3`). We will proceed with that one.


In [ ]:
ts = ts_guess2
view(ts)

In [ ]:
ea = ts.get_potential_energy() - initial.get_potential_energy()
print(f"Refined Activation Energy: {ea * 96.49} kJ/mol")

Let's confirm it's a transition state with a vibrational analysis.


In [ ]:
from ase.vibrations import Vibrations

vib = Vibrations(ts)
vib.run()
vib.summary()

One imaginary mode! Let's check it out.


In [ ]:
vib.write_mode(0)

That indeed looks like the right one! Our mission is complete.

Sella also provides an option to carry out an IRC analysis to confirm the TS connects the proposed reactants and products, but we will forego doing so here for the sake of time.
